In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim


In [ ]:
SHAPE = 20
NREALS = 30
NFREQS = 40
NLOUDEST = 10
NPSRS = 40


# Construct Parameter Space

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)

In [ ]:
param_names = pspace.param_names
num_pars = len(param_names)
pars = 0.5*np.ones(num_pars)

In [ ]:
def vary_parameter(
        target_param,    # the name of the parameter, has to exist in `param_names`
        params_list = [0.0,  0.5, 1.0],  # the values we'll check
        pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None),
        nreals=NREALS, nfreqs=NFREQS,
        pars=None, save_dir=None, 
        get_ds=False, #npsrs=NPSRS, sigma=SIGMA, nskies=NSKIES,
        ):
    # get the parameter names from this library-space
    param_names = pspace.param_names
    num_pars = len(pspace.param_names)
    print(f"{num_pars=} :: {param_names=}")

    # choose each parameter to be half-way across the range provided by the library
    if pars is None:
        pars = 0.5 * np.ones(num_pars) 
    str_pars = str(pars).replace(" ", "_").replace("[", "").replace("]", "")
    # Choose parameter to vary
    param_idx = param_names.index(target_param)

    data = []
    params = []
    dsdat = []
    for ii, par in enumerate(params_list):
        pars[param_idx] = par
        print(f"{ii=}, {pars=}")
        # _params = pspace.param_samples[0]*pars
        _params = pspace.normalized_params(pars)
        params.append(_params)
        # construct `sam` and `hard` instances based on these parameters
        sam, hard = pspace.model_for_params(_params, pspace.sam_shape)
        if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
            hard_name = 'Fixed Time'
        elif isinstance(hard, holo.hardening.Hard_GW):
            hard_name = 'GW Only'
        # run this model, retrieving binary parameters and the GWB
        _data = holo.librarian.run_model(sam, hard, nreals, nfreqs, 
                                        gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
        data.append(_data)
        if detstats:
            _dsdat = detstats.detect_pspace_model(
                _data['fobs_cents'], _data['hc_ss'], _data['hc_bg'], NPSRS, SIGMA, NSKIES)
            dsdat.append(_dsdat)
    if save_dir is not None:
        str_shape = str(sam.shape).replace(", ", "_").replace("(", "").replace(")", "")
        filename = save_dir+'/%s_p%s_s%s.npz' % (target_param, str_pars, str_shape)
        np.savez(filename, data=data, params=params, hard_name=hard_name, shape=sam.shape, target_param=target_param )
        print('saved to %s' % filename)

    return (data, params, dsdat)

# Calibrate PTA

In [ ]:
rv = vary_parameter('hard_time', params_list=[0.5])
data = rv[0][0]

# calculate dur, cad, dfobs
fobs_edges = data['fobs_edges']
fobs_cents = data['fobs_cents']
dur = 1/fobs_cents[0]
hifr = NFREQS/dur
cad = 1.0 / (2 * hifr)

In [ ]:
NPSRS = 40
SIGMA = 4.05e-6
NSKIES = 25

# build PTA
print('Building pulsar timing array.')
sum_median=0
sum_mean=0
count=30
for ii in range(count):
    phis = np.random.uniform(0, 2*np.pi, size = NPSRS)
    thetas = np.random.uniform(np.pi/2, np.pi/2, size = NPSRS)
    # sigmas = np.ones_like(phis)*sigma
    psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=SIGMA,
                    phi=phis, theta=thetas)

    dp_bg = detstats.detect_bg_pta(psrs, fobs_cents, cad, data['hc_bg'])
    # print(f"{np.median(dp_bg)=}, {np.mean(dp_bg)=}, {np.std(dp_bg)=}")
    sum_median += np.median(dp_bg)
    sum_mean += np.mean(dp_bg)
print(f"average median={sum_median/count}, average mean={sum_mean/count}")

# Run Many Hard Times

In [ ]:
params_list = np.linspace(0,1,6)
print(params_list)
data, params, dsdat = vary_parameter('hard_time', params_list=params_list)

In [ ]:
print(dsdat[1]['ev_ss'].shape)
print(data[1]['fobs_cents'].shape)
print(dsdat[1]['dp_bg'].shape)
print(dsdat[1]['dp_ss'].shape)

In [ ]:
# def_draw_ev(ax):

def draw_skies_vs_bg(ax, skies_ss, dp_bg, label=None,
                     color='k', mean=True):
    xx = dp_bg # shape (R,)
    if mean:
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    else: 
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    yerr = np.std(skies_ss, axis=-1) # shape (R,)

    hh, = ax.errorbar(xx, yy, yerr, color=color,
                linestyle='', capsize=3, marker='o', alpha=0.5)
    return hh


# def plot_evss_vs_dpbg(dsdat):
#     colors = cm.rainbow_r(np.linspace(0, 1, len(dsdat)))

#     fig, ax = plot.figax(xlabel='Background Detection Probability', 
#                          ylabel='$\langle$ Single Source Detections $\\rangle_\mathrm{skies}$')
#     for ii, ds in enumerate(dsdat):
#         draw_skies_vs_bg(ax, ds['ev_ss'], ds['dp_bg'], color=colors[ii])
#     # xx = fobs_cents*YR
#     # yy = dsdat['ev_ss']

#     return fig

def plot_dpss_vs_dpbg(dsdat, params, target_param='hard_time', use_ev=True):
    colors = cm.rainbow_r(np.linspace(0, 1, len(dsdat)))

    fig, ax = plot.figax(xlabel='Background Detection Probability', 
                         ylabel='$\langle$ Single Source Detections Probability $\\rangle_\mathrm{skies}$')
    handles = []
    for ii, ds in enumerate(dsdat):
        label = params[ii][target_param]
        detss = ds['ev_ss'] if use_ev else ds['dp_ss']
        hh = draw_skies_vs_bg(ax, detss, ds['dp_bg'], color=colors[ii], label=label)
        handles.append(hh)
    ax.legend(handles=handles, loc = 'upper left', title='$\tau_\mathrm{hard}$')
    # xx = fobs_cents*YR
    # yy = dsdat['ev_ss']

    return fig

fig1 = plot_dpss_vs_dpbg(dsdat, params, use_ev=True)
fig2 = plot_dpss_vs_dpbg(dsdat, params, use_ev=False)